In [ ]:
import csv 
import requests
import wikipediaapi
import classes as cs
import json
import networkx as nx
import graph_networkx as gnx
import nx_altair as nxa
import pandas as pd
import altair as alt
import network_functions as ntf 
from networkx.readwrite import json_graph

# Testing file: so that I don't have to contstantly rerun

In [ ]:
#ntf.parse_playlist('https://open.spotify.com/playlist/4SS7ARjCKiJmtqthv54dAG?si=agryaLlaRMWVSaO0EBCLWg&pi=u-UAcbEZPCQ_S0')



In [ ]:
#graph = gnx.createMyGraph()
#list = gnx.createMyGraph()
list = ntf.load_example_graph()


#print(graph.nodes)
#returns info --  print(mygraph.nodes['Katy Perry'])

graph = list[0]
lookup_dict = list[1]
playlist_name = list[2]



In [ ]:
graph  = ntf.get_shortest_graph(graph,'Chappell Roan','Babygirl')

for n in graph.nodes():
        graph.nodes[n]['id']=str(n)

position = nx.spring_layout(graph)

nodelayer = nxa.draw_networkx_nodes(graph,pos=position)
edgeslayer = nxa.draw_networkx_edges(graph,pos=position,width=5,edge_color="#2CDA9D")
n1=nodelayer.mark_circle(size=300,opacity=1).encode(color=alt.Color('type:N',scale=alt.Scale(range=["#CE1483",'#3772FF'])),tooltip=['id:N'])
text = nodelayer.mark_text(color='black',dx=50,fontSize=13,fontWeight="bold").encode(
    text='id:N'
    )

(edgeslayer+n1+text)



In [ ]:

color_map = []
for node in graph:
    values = graph.nodes[node]
    if values['type'] == 'artist':
        color_map.append('red') #artist
    else: 
        color_map.append('pink') #genre

nx.draw_networkx(graph,with_labels=False,node_color=color_map,node_size=20) #.interactive()

In [ ]:
for n in graph.nodes():
        graph.nodes[n]['id']=str(n)

def draw_network_old(graph):
        return(nxa.draw_networkx(graph).encode(tooltip=['id:N'],fill='type:N').properties(width=500,height=500,title='Top 100 Playlist Graph').interactive())


ntf.draw_network(graph,False,100)

In [ ]:
def draw_network(graph,labels=True,size_v=300):        #labels is true or false
    position = nx.spring_layout(graph)

    nodelayer = nxa.draw_networkx_nodes(graph,pos=position)
    edgeslayer = nxa.draw_networkx_edges(graph,pos=position)
    n1=nodelayer.mark_circle(size=size_v,opacity=1).encode(color='type:N',tooltip=['id:N'])
    text = nodelayer.mark_text(color='black').encode(
        text='id:N'
        )
    if labels:
        return(edgeslayer+n1+text)
    return((edgeslayer+n1).interactive())
draw_network(graph,False,100).properties(width=500,height=500)



In [ ]:
graph.nodes['uk pop']

In [ ]:
nx.shortest_path(graph,source='uk pop',target='vancouver indie')

In [ ]:
#here I want to get all artists & genres
def get_genre_artists(graph):
    '''
    Gets easy to parse lists of all the artists in a graph and all the genres.

    Returns the lists in a dict with keys 'artists' and 'genres'

    PARAMETERS
    ----------
    graph: networkx graph
        graph with nodes and edges, where every node has attributes id and type
    
    RETURNS
    -------
    dict: key string, value list
        dict with keys 'artists' & 'genres' with lists of all artists & genres respectively
    
    '''
    artist_list = []
    genre_list = []

    for n in graph.nodes():
        if graph.nodes[n]['type']=='artist':
            artist_list.append(n)
        elif graph.nodes[n]['type']=='genre':
            genre_list.append(n)
        else:
            print(n)
    return({'artists':artist_list,'genres':genre_list})

get_genre_artists(graph)

In [ ]:
#def filter_edge
def get_focus_graph(graph,name):
        neighbors = [n for n in graph.neighbors(name)]

        def filter_node(n1):
            if n1 ==name: 
                return True
            return (n1 in neighbors)
        
        sub_graph = nx.subgraph_view(graph,filter_node)
        return sub_graph


flor = get_focus_graph(graph,'flor')
draw_network(flor,labels=True,size_v=1000)

In [ ]:
#nx.shortest_path(graph,source='Daft Punk',target='Daddy Yankee')
def get_shortest_graph(graph,name,end):
        try:
            neighbors = nx.shortest_path(graph,source=name,target=end) #THIS THROWS THE ERROR

            def filter_node(n1):
                if n1 ==name: 
                    return True
                return (n1 in neighbors)
            
            sub_graph = nx.subgraph_view(graph,filter_node)
            return(sub_graph)
        except:
            return None
        # position = nx.kamada_kawai_layout(sub_graph)

        # nodelayer = nxa.draw_networkx_nodes(sub_graph,pos=position)
        # edgeslayer = nxa.draw_networkx_edges(sub_graph,pos=position)
        # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
        # text = nodelayer.mark_text(color='black').encode(
        #     text='id'
        #     ).properties(title=f"From {name} to {end}")
        # return(edgeslayer+n1+text)

short = get_shortest_graph(graph,'uk pop','vancouver indie') 
draw_network(short,labels=True,size_v=1000)

In [ ]:
#print(graph.nodes['Daft Punk'])

In [ ]:
def search_dead(graph,lookup):  ###NEED TO TEST LATER COULD JUST BE ISSUE BECAUSE NO DEAD PEOPLE
    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre': #if its a genre its not dead
            return False
        if lookup[n1]['died']== None:
                return False
        elif lookup[n1]['died']=='alive':
                return False
        return True
    sub_graph = nx.subgraph_view(graph,filter_node)
    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        #similar helper function to one above, returns true or false if the node should be included
        #filters to include the nodes from sub_graph and their neighbors
        if n1 in (sub_graph.nodes): 
            return True
        return (n1 in nbrs)

    return nx.subgraph_view(graph,filter_genres)


    # nodelayer = nxa.draw_networkx_nodes(and_genres,pos=position)
    # edgeslayer = nxa.draw_networkx_edges(and_genres,pos=position)
    # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
    # text = nodelayer.mark_text(color='black').encode(
    #     text='id:N' 
    #     ).properties(title=f"Dead artists")
    # return(n1+text) ###SKYELER MAKE A CHANGE-- IF NO NODES, RETURN NONE


dead=search_dead(graph,lookup_dict)
draw_network(dead,labels=True,size_v=1000)

In [ ]:
#WRITE MY TESTS
print(json_graph.node_link_data(graph))

#save my graph
# thing = json_graph.node_link_data(graph)
# fw = open('test_graph_data',"w")
# dumped_json_cache = json.dumps(thing)
# fw.write(dumped_json_cache)
# fw.close() 

# #save my lookup 
# second = open('test_lookup_data',"w")
# dumped = json.dumps(lookup_dict)
# second.write(dumped)
# second.close()




In [ ]:
def load_example_graph(graph_name='test_graph_data',dict_name='test_lookup_data'): ###SKYELER NEED TO FIX MY EXAMPLE GRAPH!!!
    ex_file = open(graph_name,"r")
    graph_contents = ex_file.read()
    graph_dict = json.loads(graph_contents)
    ex_file.close()

    dict_file = open(dict_name,"r")
    lookup_cont = dict_file.read()
    lookup = json.loads(lookup_cont)
    dict_file.close()
    return [nx.node_link_graph(graph_dict),lookup,"Skyeler's Current Tunes"]

print(load_example_graph()[1])

In [ ]:
def search_alive(graph,lookup): 
    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre': #if its a genre its not dead
            return False
        if lookup[n1]['died'] is None:
             return False
        elif lookup[n1]['died']=='alive':
                return True
        return False
    
    sub_graph= nx.subgraph_view(graph,filter_node)
    print(sub_graph)
    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors 

    def filter_genres(n1):
        if n1 in sub_graph.nodes: 
            return True
        return (n1 in nbrs)

    return nx.subgraph_view(graph,filter_genres)

alive = search_alive(graph,lookup_dict)
draw_network(alive,labels=True,size_v=200)

In [ ]:
def search_from(graph,place,lookup): ###DOUBLES AS YEAR OF BIRTH

    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre':
            return False
        if lookup[n1]['birth']==None:
            return False
        elif place.lower() in lookup[n1]['birth'].lower(): #if its a genre its not dead
            return True
        return False
    
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        nbrs += [n for n in graph.neighbors(node)]

    def filter_genres(n1):
        if n1 in (sub_graph.nodes): 
            return True
        elif graph.nodes[n1]['type']!='genre':
            return False
        return (n1 in nbrs)
    

    return nx.subgraph_view(graph,filter_genres)




england = search_from(graph,'california',lookup_dict)
old = search_from(graph,'1997',lookup_dict)
draw_network(old,labels=True,size_v=200)|draw_network(england,labels=True,size_v=200)

In [ ]:
def search_occupations(graph,job,lookup): 

    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre':
            return False
        if lookup[n1]['occupations']==None:
            return False
        elif job.lower().strip() in lookup[n1]['occupations']: #if its a genre its not dead
            return True
        return False
    
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        if n1 in (sub_graph.nodes): 
            return True
        elif graph.nodes[n1]['type']!='genre':
            return False
        return (n1 in nbrs)
    
    

    return nx.subgraph_view(graph,filter_genres)


    # position = nx.kamada_kawai_layout(and_genres)

    # nodelayer = nxa.draw_networkx_nodes(and_genres,pos=position).properties(height=500,width=500)
    # edgeslayer = nxa.draw_networkx_edges(and_genres,pos=position).properties(height=500,width=500)
    # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
    # text = nodelayer.mark_text(color='black').encode(
    #     text='id:N'
    #     ).properties(title=f"Artists with a {job} career")
    # return(edgeslayer+n1+text)

poem_girls = search_occupations(graph,'vocals',lookup_dict)
draw_network(old,labels=True,size_v=200)

In [ ]:
def search_instruments(graph,instru,lookup): 

    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre':
            return False
        if lookup[n1]['instruments']==None:
            return False
        elif instru.lower().strip() in lookup[n1]['instruments']: #if its a genre its not dead
            return True
        return False
    
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        if n1 in (sub_graph.nodes): 
            return True
        elif graph.nodes[n1]['type']!='genre':
            return False
        return (n1 in nbrs)
    
    return nx.subgraph_view(graph,filter_genres)


vocals = search_instruments(graph,'vocals',lookup_dict)
draw_network(vocals,labels=True,size_v=200)

In [ ]:
# def calculate_avg_popularity(graph,lookup):
#     count = 0
#     sum = 0

#     for n in graph.nodes:
#         if graph.nodes[n]['type']!='genre':
#             sum +=graph.nodes[n]['popularity']
#             count+=1
#     return (sum/count)

def calculate_avg_popularity(lookup):
    count = 0
    sum=0
    for n in lookup.keys():
        sum +=lookup[n]['popularity']
        count+=1
    return (sum/count)

print(calculate_avg_popularity(lookup_dict))

In [ ]:
def filter_popularity(graph,pop,lookup): #greater than

    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre':
            return False
        if lookup[n1]['popularity']==None:
            return False
        elif int(lookup[n1]['popularity'])>=int(pop): #if its a genre its not dead
            return True
        return False
    
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        if n1 in (sub_graph.nodes): 
            return True
        elif graph.nodes[n1]['type']!='genre':
            return False
        return (n1 in nbrs)
    

    return nx.subgraph_view(graph,filter_genres)


poppy = filter_popularity(graph,'60',lookup_dict)
ntf.draw_network(poppy,labels=True,size_v=400).interactive()


In [ ]:
#Let's get your genres 
def top_genres_bar(graph):
    genres = []

    for gnode in graph.nodes:
        if graph.nodes[gnode]['type']=='genre':
            genres.append([gnode,len([n for n in graph.neighbors(gnode)])])

    genre_popularity = pd.DataFrame(genres,columns = ['Genres',"Count of Artists"])
    #genre_popularity

    return alt.Chart(genre_popularity).mark_bar().encode(
        x=alt.X('Count of Artists:Q'),
        y=alt.Y('Genres:N',
                sort='-x')).transform_window(
        rank='rank(Neighbors)',
        #sort=[alt.SortField('Neighbors', order='descending')]
    ).transform_filter(
        (alt.datum.rank < 10)
    ).properties(
        title='Your top genres'
    )

top_genres_bar(graph)

In [ ]:
data = ntf.load_example_graph()
print(data)
mygraph = data[0]
# lookup = data[1]
print(mygraph.nodes['Jensen McRae'])
#ntf.draw_network(mygraph)

In [ ]:
def get_most_popular_artist(lookup):
    '''
    Return the most popular artist's name, based on spotify's popularity score. 

    PARAMETERS
    ----------
    lookup: dict
        teehee

    RETURNS
    -------
    list
        names of the most popular artist.
    '''

    pop = 0
    persons = []
    for artist in lookup.keys():
        if lookup[artist]['popularity']==pop:
            persons.append(artist)
        elif lookup[artist]['popularity']>pop:
            pop = lookup[artist]['popularity']
            persons = [artist]
    return persons

get_most_popular_artist(lookup_dict)

print(lookup_dict['Maisie Peters']['img_info'][1]) #height, width, url

In [ ]:
def get_all_occupations(lookup):
    jobs = []
    for item in lookup.keys():
        for occ in lookup[item]['occupations']:
            occ = occ.replace(',','').lower()
            if occ not in jobs:
                jobs.append(occ)
    return jobs

print(get_all_occupations(lookup_dict))

def get_all_instruments(lookup):
    inst = []
    for item in lookup.keys():
        for occ in lookup[item]['instruments']:
            occ = occ.replace(',','').lower()
            if occ not in inst:
                inst.append(occ)
    return inst

print(get_all_instruments(lookup_dict))

In [ ]:
alt.Chart(pd.DataFrame([''],columns=['Data'])).encode(x='No Path Available:Q').mark_circle()

In [ ]:
import urllib.request 
from PIL import Image 

urllib.request.urlretrieve( 
  'https://i.scdn.co/image/ab676161000051748b521134ae0ba3f60aab6811', 
   "cute.png") 
  
img = Image.open("cute.png") 
img